# Spell Checker From Scratch

## Importing Necessary Libraries

In [1]:
import re
from collections import Counter

## Implementing Spell Checker Class

In [2]:
class SpellChecker(object):
    def __init__(self, txt_file_path):
        with open(txt_file_path, 'r') as f:
            data = f.read()
        self.words = re.findall(r'\w+', data.lower())
        self.vocabulary = set(self.words)
        self.word_count_dict = Counter(self.words)
    
    def delete(self, word):
        return [word[:i] + word[i+1:] for i in range(0, len(word))]
    
    def replace(self, word):
        alphabet = "abcdefghijklmnopqrstuvwxyz"
        return [word[:i] + letter + word[i+1:] for i in range(len(word)) for letter in alphabet]
    
    def insert(self, word):
        alphabet = "abcdefghijklmnopqrstuvwxyz"
        return [word[:i] + letter + word[i:] for i in range(len(word) + 1) for letter in alphabet]
    
    def edit_one_letter(self, word):
        edit_one_set = set()
        edit_one_set.update(self.insert(word))
        edit_one_set.update(self.delete(word))
        edit_one_set.update(self.replace(word))
        return edit_one_set
    
    def edit_two_letters(self, word):
        edit_two_set = set()
        edit_one_set = self.edit_one_letter(word)
        for w in edit_one_set:
            edit_two_set.update(self.edit_one_letter(w))
        return edit_two_set
    
    
    def levenshtein_distance(self, suggestion, target):
        m = len(suggestion)
        n = len(target)
        distance_matrix = [[0] * (n + 1) for i in range(m + 1)]  

        # initial distance matrix
        for i in range(1, m + 1):
            distance_matrix[i][0] = i
        for j in range(1, n + 1):
            distance_matrix[0][j] = j
    
        for j in range(1, n + 1):
            for i in range(1, m + 1):
                if suggestion[i - 1] == target[j - 1]:
                    cost = 0
                else:
                    cost = 1
                distance_matrix[i][j] = min(distance_matrix[i - 1][j] + 1,        # deletion
                                            distance_matrix[i][j - 1] + 1,        # insertion
                                            distance_matrix[i - 1][j - 1] + cost) # substitution   

        return distance_matrix[m][n]

    
    def auto_correct(self, word):
        if word in self.vocabulary:
            print(f"The word {word} spelling is correct.")
        else:
            total_word_count = float(sum(self.word_count_dict.values()))
            word_probs_dict = {word : self.word_count_dict[word] / total_word_count for word in self.word_count_dict.keys()}
            all_possible_candidates = self.edit_one_letter(word).union(self.edit_two_letters(word))
            meaningful_words = [candidate for candidate in all_possible_candidates if candidate in self.vocabulary]
            word_cost_dict = {candidate:self.levenshtein_distance(candidate, word) for candidate in meaningful_words}
            suggestions = {candidate:word_probs_dict[candidate] for candidate in word_cost_dict.keys() if word_cost_dict[candidate] == min(word_cost_dict.values())}
            return sorted(suggestions.items(), key=lambda item: item[1], reverse=True)

## Print Results

In [3]:
sc = SpellChecker('./text.txt')
sc.auto_correct('ham')

[('him', 0.0057977269457632),
 ('am', 0.0021915311945523426),
 ('had', 0.0016880065218433798),
 ('has', 0.0004028197381671702),
 ('ha', 0.00022059176137725987),
 ('harm', 0.0001342732460557234),
 ('hap', 5.754567688102431e-05),
 ('dam', 4.795473073418693e-05),
 ('hat', 4.3159257660768235e-05),
 ('hag', 2.8772838440512156e-05),
 ('hum', 1.4386419220256078e-05),
 ('hay', 9.590946146837385e-06),
 ('ram', 9.590946146837385e-06),
 ('kam', 4.795473073418693e-06),
 ('hams', 4.795473073418693e-06),
 ('hai', 4.795473073418693e-06),
 ('sham', 4.795473073418693e-06)]